In [1]:
!pip install requests pyyaml beautifulsoup4 langchain faiss-gpu


Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 10.9 MB/s eta 0:00:0000:0100:01


In [6]:
import sys
import os

# Füge das Verzeichnis hinzu, in dem das Python-Skript sich befindet
sys.path.append(os.path.abspath("../code/chain_server"))

# Importiere die RAGChain-Klasse
from chain_server.rag_chain_with_download import RAGChain

# Erstelle eine Instanz von RAGChain
rag_chain = RAGChain()

# Führe die PICO-Suche durch und speichere die Ergebnisse in der Datenbank
search_query = "Chronic headache AND Aspirin treatment"  # Beispiel für eine PICO-Suche
rag_chain.run_pico_search(search_query)

print("PICO-Suche abgeschlossen und Ergebnisse verarbeitet.")

ModuleNotFoundError: No module named 'chain_server.rag_chain_with_download'